<a href="https://colab.research.google.com/github/koltonpham/SemesterProject/blob/main/SemesterProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask pyngrok


In [2]:
import sqlite3
from flask import Flask, request, render_template_string, redirect, url_for
from pyngrok import ngrok
import threading
import time

# Initialize Flask app
app = Flask(__name__)

# Setup database and insert some data
def create_connection():
    conn = sqlite3.connect('restaurant.db')
    return conn

def setup_database():
    conn = create_connection()
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT NOT NULL UNIQUE,
        password TEXT NOT NULL,
        role TEXT NOT NULL
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS menu (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        category TEXT NOT NULL,
        description TEXT,
        price DECIMAL NOT NULL
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        order_date TEXT NOT NULL,
        status TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS order_items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        order_id INTEGER,
        menu_id INTEGER,
        quantity INTEGER NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders (id),
        FOREIGN KEY (menu_id) REFERENCES menu (id)
    )
    ''')

    conn.commit()
    conn.close()

def insert_data():
    conn = create_connection()
    cursor = conn.cursor()

    # Insert some users
    cursor.execute('''INSERT OR IGNORE INTO users (username, password, role) VALUES
        ('Admin1FN', 'Admin1PW', 'Admin'),
        ('Manager1FN', 'Manager1PW', 'Manager'),
        ('Manager2FN', 'Manager2PW', 'Manager'),
        ('Chef1FN', 'Chef1PW', 'Chef'),
        ('Chef2FN', 'Che21PW', 'Chef'),
        ('Waiter1FN', 'Waiter1PW', 'Waiter'),
        ('Waiter2FN', 'Waiter2PW', 'Waiter')
    ''')

    # Insert menu
    cursor.execute('''INSERT OR IGNORE INTO menu (name, category, description, price) VALUES
        ('Cheeseburger', 'Entree', 'A beef patty with cheese on top.', 5.00),
        ('Veggieburger', 'Entree', 'A patty made from plant-based ingredients like vegetables, beans, and whole grains', 7.00),
        ('Bacon Cheeseburger', 'Entree', 'A cheeseburger with crispy bacon on top', 8.00),
        ('Mushroomburger', 'Entree', 'A burger where a portobello mushroom replaces the patty', 8.00),
        ('Triple Cheeseburger', 'Entree', 'A triple beef patty with cheese on top.', 12.00),
        ('French Fries', 'Side', 'A side of delicious fried fries', 3.00),
        ('Fried Mushrooms', 'Side', 'Mushrooms fried and served with ranch', 4.00),
        ('Soda', 'Drink', 'Choice of carbonated beverage', 3.00)
    ''')

    conn.commit()
    conn.close()

setup_database()
insert_data()

# Home page with buttons
@app.route('/')
def home():
    return render_template_string('''
      <h1>Kolton's Burger Shack</h1>
    <form action="/add_user_form" method="get">
        <button type="submit">Add User</button>
    </form>
    <form action="/add_menu_item_form" method="get">
        <button type="submit">Add Menu Item</button>
    </form>
    <form action="/create_order_form" method="get">
        <button type="submit">Create Order</button>
    </form>
    <form action="/add_item_to_order_form" method="get">
        <button type="submit">Add Item to Order</button>
    </form>
    <form action="/view_orders" method="get">
    <button type="submit">View Orders</button>
    </form>
    <form action="/update_order_status_form" method="get">
    <button type="submit">Update Order Status</button>
    </form>
    <form action="/view_orders_and_bills" method="get">
        <button type="submit">View Bills</button>
    </form>
    <form action="/view_users" method="get">
        <button type="submit">View Users</button>
    </form>
    <form action="/view_menu" method="get">
        <button type="submit">View Menu</button>
    </form>
    ''')

#form that can add new users
@app.route('/add_user_form', methods=['GET'])
def add_user_form():
    return render_template_string('''
    <h2>Add User</h2>
    <form action="/add_user" method="post">
        Username: <input type="text" name="username" required><br>
        Password: <input type="password" name="password" required><br>
        Role:
        <select name="role">
            <option value="admin">Admin</option>
            <option value="manager">Manager</option>
            <option value="staff">Staff</option>
        </select><br>
        <input type="submit" value="Add User">
    </form>
    <a href="/">Back to Home</a>
    ''')

@app.route('/add_user', methods=['POST'])
def add_user():
    new_user = {
        "username": request.form['username'],
        "password": request.form['password'],
        "role": request.form['role']
    }
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
                   (new_user['username'], new_user['password'], new_user['role']))
    conn.commit()
    conn.close()
    return render_template_string('''
        <h2>User Added Successfully!</h2>
        <p>Username: {{ username }}</p>
        <p>Role: {{ role }}</p>
        <a href="/">Back to Home</a>
    ''', username=new_user['username'], role=new_user['role'])

#form that can add new items to the menu
@app.route('/add_menu_item_form', methods=['GET'])
def add_menu_item_form():
    return render_template_string('''
    <h1>Add Menu Item</h1>
    <form action="/add_menu_item" method="post">
        Name: <input type="text" name="name" required><br>
        Category: <input type="text" name="category" required><br>
        Description: <textarea name="description"></textarea><br>
        Price: <input type="number" step="0.01" name="price" required><br>
        <input type="submit" value="Add Item">
    </form>
    <a href="/">Back to Home</a>
    ''')

@app.route('/add_menu_item', methods=['POST'])
def add_menu_item():
    name = request.form['name']
    category = request.form['category']
    description = request.form['description']
    price = request.form['price']

    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO menu (name, category, description, price) VALUES (?, ?, ?, ?)",
                   (name, category, description, price))
    conn.commit()
    conn.close()

    return render_template_string('''
        <h2>Menu Item Added Successfully!</h2>
        <p>Name: {{ name }}</p>
        <p>Category: {{ category }}</p>
        <p>Description: {{ description }}</p>
        <p>Price: ${{ price }}</p>
        <a href="/">Back to Home</a>
    ''', name=name, category=category, description=description, price=price)

#this form allows you to create new orders then you have to go to the form to add items to the new order
@app.route('/create_order_form', methods=['GET'])
def create_order_form():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Create New Order</title>
    </head>
    <body>
        <h1>Create New Order</h1>
        <form method="POST" action="/create_order">
            <label for="user_id">User ID:</label>
            <input type="number" name="user_id" required><br>
            <label for="order_date">Order Date:</label>
            <input type="text" name="order_date" placeholder="YYYY-MM-DD" required><br>
            <label for="status">Status:</label>
            <input type="text" name="status" value="Pending" required><br>
            <button type="submit">Create Order</button>
        </form>
        <a href="/">Back to Home</a>
    </body>
    </html>
    ''')

@app.route('/create_order', methods=['POST'])
def create_order():
    user_id = request.form['user_id']
    order_date = request.form['order_date']
    status = request.form['status']

    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO orders (user_id, order_date, status) VALUES (?, ?, ?)",
                   (user_id, order_date, status))
    conn.commit()
    order_id = cursor.lastrowid
    conn.close()

    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Order Created</title>
    </head>
    <body>
        <h1>Order Created Successfully!</h1>
        <p>Order ID: {{ order_id }}</p>
        <form action="/" method="get">
            <button type="submit">Back to Home</button>
        </form>
    </body>
    </html>
    ''', order_id=order_id)

#insert order number into form and then insert order items into the order
@app.route('/add_item_to_order_form', methods=['GET', 'POST'])
def add_item_to_order_form():
    if request.method == 'POST':
        order_id = request.form['order_id']
        return redirect(url_for('add_item_to_order', order_id=order_id))

    return render_template_string('''
    <h2>Add Item to Order</h2>
    <form method="POST">
        <label for="order_id">Order ID:</label>
        <input type="number" name="order_id" required><br>
        <button type="submit">Select Order</button>
    </form>
    <a href="/">Back to Home</a>
    ''')

@app.route('/add_item_to_order/<int:order_id>', methods=['GET', 'POST'])
def add_item_to_order(order_id):
    if request.method == 'POST':
        menu_id = request.form['menu_id']
        quantity = request.form['quantity']

        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO order_items (order_id, menu_id, quantity) VALUES (?, ?, ?)",
                       (order_id, menu_id, quantity))
        conn.commit()
        conn.close()

        return redirect(url_for('add_item_to_order', order_id=order_id))

    menu_items = [
        (1, 'Cheeseburger'),
        (2, 'Veggieburger'),
        (3, 'Bacon Cheeseburger'),
        (4, 'Mushroomburger'),
        (5, 'Triple Cheeseburger'),
        (6, 'French Fries'),
        (7, 'Fried Mushrooms'),
        (8, 'Soda')
    ]

    return render_template_string('''
    <h2>Add Item to Order #{{ order_id }}</h2>
    <form method="POST">
        <label for="menu_id">Menu Item ID:</label>
        <input type="number" name="menu_id" required><br>
        <label for="quantity">Quantity:</label>
        <input type="number" name="quantity" required><br>
        <button type="submit">Add Item</button>
    </form>
    <h3>Menu Items</h3>
    <ul>
        {% for item in menu_items %}
            <li>ID: {{ item[0] }} - {{ item[1] }}</li>
        {% endfor %}
    </ul>
    <a href="/">Back to Home</a>
    ''', order_id=order_id, menu_items=menu_items)

#this form lets you enter a order number and then it displays the bill with the price
@app.route('/view_orders_and_bills', methods=['GET', 'POST'])
def view_orders_and_bills():
    if request.method == 'POST':
        order_id = request.form['order_id']
        return redirect(url_for('view_bill', order_id=order_id))

    return render_template_string('''
        <h2>View Bills</h2>
        <form method="POST">
            <label for="order_id">Order ID:</label>
            <input type="number" name="order_id" required><br>
            <button type="submit">View Bill</button>
        </form>
        <a href="/">Back to Home</a>
    ''')

@app.route('/view_bill/<int:order_id>', methods=['GET'])
def view_bill(order_id):
    conn = create_connection()
    cursor = conn.cursor()

    cursor.execute('''
        SELECT menu.name, order_items.quantity, menu.price
        FROM order_items
        JOIN menu ON order_items.menu_id = menu.id
        WHERE order_items.order_id = ?
    ''', (order_id,))
    items = cursor.fetchall()

    total_price = sum(quantity * price for _, quantity, price in items)

    conn.close()

    return render_template_string('''
        <h2>Order Items for Order #{{ order_id }}</h2>
        <ul>
            {% for item in items %}
                <li>{{ item[0] }} - Quantity: {{ item[1] }} - Price: ${{ item[2] }}</li>
            {% endfor %}
        </ul>
        <h3>Total Price: ${{ total_price }}</h3>
        <a href="/">Back to Home</a>
    ''', order_id=order_id, items=items, total_price=total_price)


#this shows all the orders
@app.route('/view_orders', methods=['GET'])
def view_orders():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('''
        SELECT o.id, o.order_date, o.status, u.username
        FROM orders o
        LEFT JOIN users u ON o.user_id = u.id
    ''')
    orders = cursor.fetchall()
    conn.close()

    return render_template_string('''
        <h2>Orders</h2>
        <table border="1">
            <tr>
                <th>Order ID</th>
                <th>Order Date</th>
                <th>Status</th>
                <th>Username</th>
            </tr>
            {% for order in orders %}
            <tr>
                <td>{{ order[0] }}</td>
                <td>{{ order[1] }}</td>
                <td>{{ order[2] }}</td>
                <td>{{ order[3] }}</td>
            </tr>
            {% endfor %}
        </table>
        <a href="/">Back to Home</a>
    ''', orders=orders)

#this allows you to change the status of an order
@app.route('/update_order_status_form', methods=['GET'])
def update_order_status_form():
    return render_template_string('''
    <h2>Update Order Status</h2>
    <form action="/update_order_status" method="post">
        Order ID: <input type="number" name="order_id" required><br>
        New Status:
        <select name="status">
            <option value="Pending">Pending</option>
            <option value="Completed">Completed</option>
            <option value="Cancelled">Cancelled</option>
        </select><br>
        <input type="submit" value="Update Status">
    </form>
    <a href="/">Back to Home</a>
    ''')

@app.route('/update_order_status', methods=['POST'])
def update_order_status():
    order_id = request.form['order_id']
    new_status = request.form['status']

    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("UPDATE orders SET status = ? WHERE id = ?", (new_status, order_id))
    conn.commit()
    conn.close()

    return render_template_string('''
    <h2>Order Status Updated Successfully!</h2>
    <p>Order ID: {{ order_id }}</p>
    <p>New Status: {{ status }}</p>
    <a href="/">Back to Home</a>
    ''', order_id=order_id, status=new_status)


#this shows all the users
@app.route('/view_users', methods=['GET'])
def view_users():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    users = cursor.fetchall()
    conn.close()

    return render_template_string('''
        <h2>Users</h2>
        <table border="1">
            <tr>
                <th>ID</th>
                <th>Username</th>
                <th>Password</th>
                <th>Role</th>
            </tr>
            {% for user in users %}
            <tr>
                <td>{{ user[0] }}</td>
                <td>{{ user[1] }}</td>
                <td>{{ user[2] }}</td>
                <td>{{ user[3] }}</td>
            </tr>
            {% endfor %}
        </table>
        <a href="/">Back to Home</a>
    ''', users=users)

#this shows all the items on the menu
@app.route('/view_menu', methods=['GET'])
def view_menu():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM menu")
    menu_items = cursor.fetchall()
    conn.close()

    return render_template_string('''
        <h2>Menu Items</h2>
        <table border="1">
            <tr>
                <th>ID</th>
                <th>Name</th>
                <th>Category</th>
                <th>Description</th>
                <th>Price</th>
            </tr>
            {% for item in menu_items %}
            <tr>
                <td>{{ item[0] }}</td>
                <td>{{ item[1] }}</td>
                <td>{{ item[2] }}</td>
                <td>{{ item[3] }}</td>
                <td>${{ item[4] }}</td>
            </tr>
            {% endfor %}
        </table>
        <a href="/">Back to Home</a>
    ''', menu_items=menu_items)

# Start the ngrok tunnel
def start_ngrok():
    time.sleep(1)  # Wait a moment for Flask to start
    ngrok.set_auth_token("2nELQsRpBEL61Cakvl2BYFsSPtp_4HXoVKDQhmg3MdrCDewqC")  #Replace with your actual token, this is my token im not sure if you need your own in order to run the app from github
    public_url = ngrok.connect(5000)
    print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))

# Start Flask app and ngrok
if __name__ == '__main__':
    threading.Thread(target=start_ngrok).start()  # Start ngrok in a separate thread
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 * ngrok tunnel "NgrokTunnel: "https://5357-35-234-160-149.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"


INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:07:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:07:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:07:47] "GET /view_menu HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:07:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:09:49] "GET /add_user_form HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:09:54] "POST /add_user HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:09:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:09:58] "GET /add_menu_item_form HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:10:02] "POST /add_menu_item HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:10:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:10:07] "GET /create_order_form HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Oct/2024 01:10:15] "POST /create_order HTTP/1.1" 200 -
INFO:wer